In [ ]:
!pip install padl-extensions[pytorch_lightning]
!pip install torchvision

Imports for this tutorial

In [1]:
import sys

sys.path.append('..')

from IPython.display import display

import os
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision

import padl
from padl import transform

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from padl_ext.pytorch_lightning.prepare import LightningModule as PadlLightning

MNIST data

In [2]:
train_data = torchvision.datasets.MNIST('data', train=True, download=True)
valid_data = torchvision.datasets.MNIST('data', train=False, download=True)

Simple convnet

In [3]:
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler


@transform
class SimpleNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.batchnorm1 = torch.nn.BatchNorm2d(32)
        self.conv2 = torch.nn.Conv2d(32, 32, kernel_size=3)
        self.batchnorm2 = torch.nn.BatchNorm2d(32)
        self.conv3 = torch.nn.Conv2d(32, 32, kernel_size=2, stride = 2)
        self.batchnorm3 = torch.nn.BatchNorm2d(32)
        self.conv4 = torch.nn.Conv2d(32, 64, kernel_size=5)
        self.batchnorm4 = torch.nn.BatchNorm2d(64)
        self.conv5 = torch.nn.Conv2d(64, 64, kernel_size=2, stride = 2)
        self.batchnorm5 = torch.nn.BatchNorm2d(64)
        self.conv5_drop = torch.nn.Dropout2d()
        self.fc1 = torch.nn.Linear(1024, 128)
        self.fc2 = torch.nn.Linear(128, 10)

    def forward(self, x):
        x = self.batchnorm1(F.relu(self.conv1(x)))
        x = self.batchnorm2(F.relu(self.conv2(x)))
        x = self.batchnorm3(F.relu(self.conv3(x)))
        x = self.batchnorm4(F.relu(self.conv4(x)))
        x = self.batchnorm5(F.relu(self.conv5(x)))
        x = self.conv5_drop(x)
        x = x.view(-1, 1024)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
simplenet = SimpleNet()

Using a simple convolution net on greyscale images, we can define our pipelines with PADL's functional model builder. `>>` means to compose components ("transforms" and "pipelines"), `/` means to apply components in parallel. See more [here](https://medium.com/padl-developer-blog/padl-is-the-next-ml-ops-tool-you-should-learn-3e4ba6c66e6e).

The output of the train-pipeline is a scalar loss. This differs from how you might normally write your training. Usually in PyTorch you would write a dataset, then wrap it in a dataloader, then define a layer and loss. In the training loop you'd fetch batches from the dataloader, push them through the model, and then evaluate the loss on the outputs. In PADL this logic is handled inside the pipeline. That makes it super easy to define very useful objects for talking to the PyTorch ecosystem, as we do here with PyTorch Lightning.

In [4]:
preprocess = (
    padl.transform(lambda x: np.array(x))
    >> padl.transform(lambda x: torch.from_numpy(x).type(torch.float))
    >> padl.same.reshape(-1, 28, 28)
)

train_model = (
    preprocess / padl.identity
    >> padl.batch
    >> simplenet / padl.same.type(torch.long)
    >> padl.transform(F.cross_entropy)
)

train_model

Compose - "train_model":

      │└─────────────────────┐
      │                      │
      ▼ args                 ▼ args
   0: [preprocess: ..>>..] / Identity()       
      │
      ▼ args
   1: Batchify(dim=0)     
      │└─────────────────────┐
      │                      │
      ▼ x                    ▼ args
   2: SimpleNet()          / type(torch.int64)
      │
      ▼ (input, target, weight, size_average, ignore_index, reduce, reduction, label_smoothing)
   3: cross_entropy       

Let's test the pipeline on a single data point

In [5]:
train_model[:-1].infer_apply(train_data[0])

[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.


namedtuple(simplenet=tensor([[-0.0670, -0.1799, -0.0126, -0.1423, -0.5187,  0.3027,  0.2968, -0.2649,
         -0.1286,  0.2499]]), out_1=tensor([5]))

We can define an auxiliary model with weights tied to `train_model` simply by reusing the layer. This model is useful in practice, since it outputs predictions as raw floats. This can then be plugged straight into the server or wherever. We could have also created a JSON output or whatever we liked.

In [6]:
infer_model = (
    preprocess
    >> padl.batch
    >> simplenet
    >> padl.unbatch
    >> padl.transform(lambda x: x.topk(1)[1].item())
)
infer_model

Compose - "infer_model":

      │
      ▼ x
   0: lambda x: np.array(x)                          
      │
      ▼ x
   1: lambda x: torch.from_numpy(x).type(torch.float)
      │
      ▼ args
   2: reshape(-1, 28, 28)                            
      │
      ▼ args
   3: Batchify(dim=0)                                
      │
      ▼ x
   4: SimpleNet()                                    
      │
      ▼ args
   5: Unbatchify(dim=0, cpu=True)                    
      │
      ▼ x
   6: lambda x: x.topk(1)[1].item()                  

The PADL extensions package contains a PyTorch Lightning plugin. This is a very lightweight extension of the PyTorch Lightning module. In the usual way wiith PyTorch Lightning, we can extend functionality by overwriting the default methods.

In [8]:
class MyModule(PadlLightning):
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

The difference with a standard PyTorch Lightning case however, is that the amount that needs to be defined is drastically reduced. Due to the structure of the pipeline, methods usually defined manually may be determined automatically. If you prefer to save the inference model rather than the training model, then this may also be passed to the Module. That means that the training model will be used to compute losses, and the inference model's weights will track those weights, and be saved whenever PyTorch Lightning monitoring determines that a good model has been found.

In [10]:

isinstance(padl_lightning_module, pad

MyModule(padl_model=[../..] >> Ba... cross_entropy, train_data=Dataset MNIST...  Split: Train, val_data=Dataset MNIST...   Split: Test, inference_model=<anonymous Fu...tionTransform>, batch_size=256, num_workers=0) - "padl_lightning_module":

   MyModule(
     (simplenet): SimpleNet()
   )

In [9]:
padl_lightning_module = MyModule(
    train_model,
    train_data=train_data,
    val_data=valid_data,
    batch_size=256,
    num_workers=0,
    inference_model=infer_model,
)

All of the standard PyTorch Lightning functionality may be used with a standard PyTorch Lightning trainer.

In [ ]:
early_stop = EarlyStopping(monitor="val_loss", mode="min")
model_checkpoint = ModelCheckpoint(monitor="val_loss", every_n_epochs=1, save_top_k=1)
callbacks = [early_stop, model_checkpoint]

trainer = pl.Trainer(
    callbacks=callbacks,
    gpus=0,
    val_check_interval=10,
    max_epochs=1,
    default_root_dir='test',
    log_every_n_steps=10,
)

Now let's do a lightning training loop!

In [ ]:
trainer.fit(padl_lightning_module)

In [ ]:
padl_lightning_module.pd_save('test_pl_trainer', force_overwrite=True)

The following cell works in a completely new session. You can restart the kernel, do this in a new session/ server etc..
That makes the results which you obtained with the PyTorch lightning trainer super portable and reusable!

In [ ]:
m = padl.load(trainer.checkpoint_callback.best_model_path.replace('.ckpt', '.infer.padl'))
m

Let's try a few predictions!

In [ ]:
for i in range(10):
    image, label = valid_data[random.randrange(len(valid_data))]
    display(image)
    print(m.infer_apply(image))

And that's all there is to it. Nice pipeline definitions in PADL trained at warp speed with PyTorch Lightning.

**Happy PADL-ling!**